In [1]:
import random as rd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
from scipy import stats
import math

In [2]:
# Queue of Events
class EventsQueue:
    def __init__(self):
        self.globalTime = 0
        self.MEvents = []

    def QueueSize(self):
        return len(self.MEvents)

    def add_event(self, MEvent):
        count = len(self.MEvents)
        if count == 0 or MEvent.eTime >= self.MEvents[count - 1].eTime:
            self.MEvents.append(MEvent)
            return 0

        for i in range(0, count-1):
            if MEvent.eTime >= self.MEvents[i].eTime:
                if MEvent.eTime < self.MEvents[i + 1].eTime:
                    self.MEvents.insert(i + 1, MEvent)
                    return 0

    def process_next_event(self):
        if len(self.MEvents) == 0:
            return 0
        self.MEvents[0].Execute()
        self.globalTime = self.MEvents[0].eTime
        del self.MEvents[0]

In [27]:
# Discrete Event System Specification
class DEVS:
    def __init__(self):
        self.EQ = EventsQueue()
        self.GlobalTime = 0.0

        # simulation attributes
        self.customerQueue = []
        self.stats = []
        self.newId = 0
        self.serverIdle = True
        self.lastServedTime = 0  # for Idle time

    def process_next_event(self):
        self.EQ.process_next_event()
        self.GlobalTime = self.EQ.globalTime

In [3]:
# ---- Customer Statistics ----
class CustomerStat:
    def __init__(self):
        self.id = -1
        self.arrivalTime = -1
        self.serviceTime = -1
        self.interArrivalTime = 0
        self.serviceBegins = -1
        self.waitingTimeInQueue = 0
        self.serviceEnds = -1
        self.timeInSystem = -1
        self.idleTimeOfServer = 0

In [ ]:
# ---- Arrival Event ----
class ArrivalEvent:
    def __init__(self, devs, maxAngents, arrivalRateMin, arrivalRateMax, custm, verbose, threshold=None):
        self.devs = devs
        self.maxAngents = maxAngents
        self.arrivalRateMin = arrivalRateMin
        self.arrivalRateMax = arrivalRateMax
        self.custm = custm
        self.eTime = 0.0
        self.threshold = threshold
        self.verbose = verbose

    def Execute(self):
        customer = CustomerStat()
        customer.id = self.devs.newId
        customer.arrivalTime = self.eTime
        if len(self.devs.stats) > 0:
            customer.interArrivalTime = customer.arrivalTime - self.devs.stats[-1].arrivalTime

        if self.verbose:
            print("Time %d" % self.eTime, " Arrival Event of agent {0}".format(customer.id))
            print(self.devs.EQ.MEvents)
        if self.devs.newId < self.maxAngents - 1:
            next_arrival = ArrivalEvent(self.devs, self.maxAngents, self.arrivalRateMin, self.arrivalRateMax,
                                        self.custm, self.verbose, self.threshold)
            next_arrival.eTime = self.eTime + rd.randint(self.arrivalRateMin, self.arrivalRateMax)
            self.devs.EQ.add_event(next_arrival)

        # server is Free
        if (self.devs.serverIdle == True):
            self.devs.serverIdle = False
            if self.verbose:
                print("server is Busy")
            Service = ServiceEvent(self.devs, self.custm, self.verbose)
            serviceTime = self.custm.rvs()  
            customer.serviceTime = serviceTime
            customer.serviceBegins = self.eTime # current time
            Service.eTime = self.eTime + serviceTime            
            Service.id = customer.id            
            self.devs.EQ.add_event(Service)
        # server is Busy
        else:
            # increase waiting line
            self.devs.customerQueue.append(customer.id)
            if self.verbose:
                print("customerQueue = %d"%len(self.devs.customerQueue))

        self.devs.newId = self.devs.newId + 1
        self.devs.stats.append(customer)

In [28]:
# ---- Service (END) Event ----
class ServiceEvent:
    def __init__(self, devs, custm, verbose):
        self.devs = devs
        self.custm = custm
        self.eTime = 0.0
        self.id = 0
        self.verbose = verbose

    def Execute(self):
        ind = [i for i, val in enumerate(self.devs.stats) if val.id == self.id][0]
        self.devs.stats[ind].serviceEnds = self.eTime
        self.devs.stats[ind].timeInSystem = self.devs.stats[ind].serviceEnds - self.devs.stats[ind].arrivalTime
        self.devs.stats[ind].waitingTimeInQueue = self.devs.stats[ind].serviceBegins - self.devs.stats[ind].arrivalTime # 0 without queue
        self.devs.stats[ind].idleTimeOfServer = self.devs.stats[ind].serviceBegins - self.devs.lastServedTime

        if self.verbose:
            print("Time %d" % self.eTime, "Service finished")
            
        if(len(self.devs.customerQueue)>0):
            qid = self.devs.customerQueue.pop(0)
            qind = [i for i,val in enumerate(self.devs.stats) if val.id == qid][0]
            Service = ServiceEvent(self.devs, self.custm, self.verbose)
            serviceTime = custm.rvs()  
            Service.eTime = self.eTime + serviceTime
            Service.id = qid
            self.devs.stats[qind].serviceBegins = self.eTime
            self.devs.stats[qind].serviceTime = serviceTime            
            self.devs.EQ.add_event(Service)
            if self.verbose:
                print("take new customer from the queue")            
        else:
            self.devs.serverIdle = True
            if self.verbose:
                print("server is Idle (do nothing)")

        self.devs.lastServedTime = self.eTime


def run_devs(maxAngents, arrivalRateMin, arrivalRateMax, custm, verbose=False):
    # run simulation
    devs = DEVS()
    AE = ArrivalEvent(devs, maxAngents, arrivalRateMin, arrivalRateMax, custm, verbose)
    devs.EQ.add_event(AE)

    # --- SIMULATION ---
    while devs.EQ.QueueSize() > 0:
        devs.process_next_event()

    return devs

In [4]:
def avTimeInQueue(devs):
    return sum([x.waitingTimeInQueue for x in devs.stats]) / len(devs.stats)

def probToWait(devs):
    return len([x for x in devs.stats if x.waitingTimeInQueue > 0]) / len(devs.stats)

def probIdle(devs):
    return sum([x.idleTimeOfServer for x in devs.stats]) / devs.GlobalTime

def avServiceTime(devs):
    return sum([x.serviceTime for x in devs.stats]) / len(devs.stats)

def avTimeBetwArr(devs):
    return sum([x.interArrivalTime for x in devs.stats]) / (len(devs.stats) - 1)

def avTimeWhoWait(devs):
    numOfCustWhoWait = len([x for x in devs.stats if x.waitingTimeInQueue > 0])
    return sum([x.waitingTimeInQueue for x in devs.stats]) / numOfCustWhoWait if numOfCustWhoWait else 0

def avTimeInTheSystem(devs):
    return sum([x.timeInSystem for x in devs.stats]) / len(devs.stats)

In [5]:
# Доверительный интервал
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    m, se = np.mean(a), np.std(a,ddof=1)/np.sqrt(n)
    h = se * sp.stats.t._ppf((1+confidence)/2., n-1)
    return m, h

In [6]:
# Параметры системы
maxAngents = 100
arrivalRateMin = 1
arrivalRateMax = 4
service_xk = np.arange(6) + 5
service_pk = (0.1, 0.2, 0.3, 0.25, 0.1, 0.05)
custm = stats.rv_discrete(name='custm', values=(service_xk, service_pk))

In [24]:
statistics = []
threshold = 0.3

for i in range(20):
    devs = run_devs(maxAngents, arrivalRateMin, arrivalRateMax, custm)
    statistics.append(avTimeInTheSystem(devs))

m, h = mean_confidence_interval(statistics)
replications = (h * np.sqrt(len(statistics)) / threshold) ** 2

print("replications = " +str(int(replications)))
print("h = " + str(h))
print("m = " + str(m))

replications = 5537
h = 4.991751377979128
m = 240.00899999999996


In [25]:
for i in range(math.ceil(replications) - len(statistics)):
    devs = run_devs(maxAngents, arrivalRateMin, arrivalRateMax, custm)
    statistics.append(avTimeInTheSystem(devs))

m, h = mean_confidence_interval(statistics)
print("h = " + str(h))
print("m = " + str(m))
if h > threshold:
    replications = (h * np.sqrt(len(statistics)) / threshold) ** 2
    print("replications = " +str(int(replications)))

h = 0.2605441299558768
m = 239.93391477067533


In [26]:
# Автоматизировал поиск параметра с нужной точностью. Выполняется в цикле до достижения порога.
statistics = []
threshold = 0.3

for i in range(20):
    devs = run_devs(maxAngents, arrivalRateMin, arrivalRateMax, custm)
    statistics.append(avTimeInTheSystem(devs))

m, h = mean_confidence_interval(statistics)
replications = (h * np.sqrt(len(statistics)) / threshold) ** 2

print("replications = " +str(int(replications)))
print("h = " + str(h))
print("m = " + str(m))

while h > threshold:
    for i in range(math.ceil(replications) - len(statistics)):
        devs = run_devs(maxAngents, arrivalRateMin, arrivalRateMax, custm)
        statistics.append(avTimeInTheSystem(devs))

    m, h = mean_confidence_interval(statistics)
    print("h = " + str(h))
    print("m = " + str(m))
    if h > threshold:
        replications = (h * np.sqrt(len(statistics)) / threshold) ** 2
        print("replications = " +str(int(replications)))

replications = 5020
h = 4.753178124093773
m = 237.593
h = 0.2742039188271308
m = 239.93979087831107
